In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport

# visulaisation
from matplotlib.pyplot import xticks
%matplotlib inline

In [2]:
instalments_payments = pd.DataFrame(pd.read_csv('C:/Users/kalin/installments_payments.csv'))
instalments_payments

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.0,66,-1624.0,NaN,67.500,NaN
13605397,1310347,414406,0.0,47,-1539.0,NaN,67.500,NaN
13605398,1308766,402199,0.0,43,-7.0,NaN,43737.435,NaN
13605399,1062206,409297,0.0,43,-1986.0,NaN,67.500,NaN


# Traitement des données manquantes

In [3]:
# pourcentage de données manquantes de chaque variables :
count = instalments_payments.isnull().sum().sort_values(ascending=False)
percentage = ((instalments_payments.isnull().sum()/len(instalments_payments)*100)).sort_values(ascending=False)
missing_application = pd.concat([count, percentage], axis=1, keys=['Count','Percentage'])
print('Count and percentage of missing values s:')
missing_application

Count and percentage of missing values s:


,Count,Percentage
DAYS_ENTRY_PAYMENT,2905,0.021352
AMT_PAYMENT,2905,0.021352
SK_ID_PREV,0,0.000000
SK_ID_CURR,0,0.000000
NUM_INSTALMENT_VERSION,0,0.000000
NUM_INSTALMENT_NUMBER,0,0.000000
DAYS_INSTALMENT,0,0.000000
AMT_INSTALMENT,0,0.000000


In [4]:
# variables ayant plus de 50% de DM
missing_application_sup50=missing_application[ missing_application['Percentage'] > 50]
missing_application_sup50

,Count,Percentage


In [5]:
missing_application_sup50.shape

(0, 2)

In [6]:
#variables aynat des données manquantes < 50% dans la nouvelle table application_test_ssDM

instalments_payments.columns[instalments_payments.isnull().any()]

Index(['DAYS_ENTRY_PAYMENT', 'AMT_PAYMENT'], dtype='object')

In [7]:
# Remplacer en utilisant la médiane

def DM_mediane (data, var):
    median = data[var].median()
    data[var].fillna(median, inplace=True)

In [8]:
DM_mediane(instalments_payments, 'DAYS_ENTRY_PAYMENT')
DM_mediane(instalments_payments, 'AMT_PAYMENT')

In [9]:
prof = ProfileReport(instalments_payments)
prof.to_file(output_file='instalments_payments.html')

Summarize dataset:   0%|          | 0/22 [00:00<?, ?it/s]

C:\Users\kalin\anaconda3\envs\OCR5\lib\site-packages\scipy\stats\stats.py:4594: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


KeyboardInterrupt: 

# Fichier CSV

In [10]:
instalments_payments.to_csv('C:/Users/kalin/instalments_payments.csv')

# Preprocess installments_payments.csv

In [11]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category) # dummy_na =Add a column to indicate NaNs, if False NaNs are ignored
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [12]:
def installments_payments(num_rows = None, nan_as_category = True):
    ins = pd.read_csv('C:/Users/kalin/instalments_payments.csv', nrows = num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    
    return ins_agg

In [13]:
installments_payments_FE = installments_payments(num_rows = None, nan_as_category = True)
installments_payments_FE

,INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE,INSTAL_DPD_MAX,INSTAL_DPD_MEAN,INSTAL_DPD_SUM,INSTAL_DBD_MAX,INSTAL_DBD_MEAN,INSTAL_DBD_SUM,INSTAL_PAYMENT_PERC_MAX,INSTAL_PAYMENT_PERC_MEAN,INSTAL_PAYMENT_PERC_SUM,...,INSTAL_AMT_INSTALMENT_MEAN,INSTAL_AMT_INSTALMENT_SUM,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_MAX,INSTAL_AMT_PAYMENT_MEAN,INSTAL_AMT_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,2,11.0,1.571429,11.0,36.0,8.857143,62.0,1.00000,1.000000,7.000000,...,5885.132143,41195.925,3951.000,17397.900,5885.132143,41195.925,-1628.0,-2195.000000,-15365.0,7
100002,2,0.0,0.000000,0.0,31.0,20.421053,388.0,1.00000,1.000000,19.000000,...,11559.247105,219625.695,9251.775,53093.745,11559.247105,219625.695,-49.0,-315.421053,-5993.0,19
100003,2,0.0,0.000000,0.0,14.0,7.160000,179.0,1.00000,1.000000,25.000000,...,64754.586000,1618864.650,6662.970,560835.360,64754.586000,1618864.650,-544.0,-1385.320000,-34633.0,25
100004,2,0.0,0.000000,0.0,11.0,7.666667,23.0,1.00000,1.000000,3.000000,...,7096.155000,21288.465,5357.250,10573.965,7096.155000,21288.465,-727.0,-761.666667,-2285.0,3
100005,2,1.0,0.111111,1.0,37.0,23.666667,213.0,1.00000,1.000000,9.000000,...,6240.205000,56161.845,4813.200,17656.245,6240.205000,56161.845,-470.0,-609.555556,-5486.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,2,0.0,0.000000,0.0,46.0,36.285714,254.0,1.00000,1.000000,7.000000,...,7492.924286,52450.470,6605.910,12815.010,7492.924286,52450.470,-38.0,-156.285714,-1094.0,7
456252,1,3.0,0.500000,3.0,11.0,3.333333,20.0,1.00000,1.000000,6.000000,...,10069.867500,60419.205,10046.880,10074.465,10069.867500,60419.205,-2327.0,-2393.833333,-14363.0,6
456253,1,9.0,0.642857,9.0,51.0,15.142857,212.0,1.00000,0.928571,13.000000,...,4399.707857,61595.910,27.270,5575.185,4115.915357,57622.815,-1738.0,-2387.428571,-33424.0,14


In [20]:
indices_to_keep = ~installments_payments_FE.isin([np.nan, np.inf, -np.inf]).any(1)


array([[-0.16401594, -0.06955429,  0.05147678, ..., -2.13764619,
         0.47202002, -0.80800621],
       [-0.16401594, -0.16780557, -0.11397895, ...,  1.01879838,
         0.63671823, -0.5157275 ],
       [-0.16401594, -0.16780557, -0.11397895, ..., -0.77792143,
         0.13341513, -0.36958814],
       ...,
       [-0.698201  , -0.08741816, -0.04629251, ..., -2.46079842,
         0.15466141, -0.63751029],
       [-0.698201  , -0.16780557, -0.11397895, ...,  1.27768128,
         0.68819081, -0.5157275 ],
       [ 0.90435419, -0.10528203, -0.03999137, ...,  0.75582703,
         0.1282134 ,  0.82388327]])

In [17]:
installments_payments_FE.columns[installments_payments_FE[~indices_to_keep].isin([np.nan, np.inf, -np.inf]).any()]

Index(['INSTAL_PAYMENT_PERC_MAX', 'INSTAL_PAYMENT_PERC_MEAN',
       'INSTAL_PAYMENT_PERC_SUM', 'INSTAL_PAYMENT_PERC_VAR',
       'INSTAL_PAYMENT_DIFF_VAR'],
      dtype='object')

In [18]:
installments_payments_FE[~indices_to_keep][['INSTAL_PAYMENT_PERC_MAX', 'INSTAL_PAYMENT_PERC_MEAN',
       'INSTAL_PAYMENT_PERC_SUM', 'INSTAL_PAYMENT_PERC_VAR',
       'INSTAL_PAYMENT_DIFF_VAR']]

,INSTAL_PAYMENT_PERC_MAX,INSTAL_PAYMENT_PERC_MEAN,INSTAL_PAYMENT_PERC_SUM,INSTAL_PAYMENT_PERC_VAR,INSTAL_PAYMENT_DIFF_VAR
SK_ID_CURR,,,,,
100254,1.0,1.0,1.0,NaN,NaN
100683,1.0,1.0,1.0,NaN,NaN
101762,1.0,1.0,1.0,NaN,NaN
102789,1.0,1.0,1.0,NaN,NaN
103054,inf,inf,inf,NaN,2.444394e+06
...,...,...,...,...,...
455162,1.0,1.0,1.0,NaN,NaN
455220,1.0,1.0,1.0,NaN,NaN
455244,1.0,1.0,1.0,NaN,NaN


# Fichier CSV

In [14]:
installments_payments_FE.to_csv('C:/Users/kalin/installments_payments_FE.csv')